In [1]:
# ChainEats Analytics - Day 4: Advanced SQL Analytics
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

print("ChainEats Advanced Analytics - Day 4")
print("=" * 50)

# Connect to database
conn = sqlite3.connect('chaineats.db')

# ADVANCED QUERY 1: Monthly Growth Trends by Location
print("\nQUERY 1: Monthly Growth Analysis")
print("-" * 40)

query1 = """
WITH monthly_data AS (
    SELECT 
        location_id,
        city,
        year,
        month,
        monthly_revenue,
        LAG(monthly_revenue) OVER (PARTITION BY location_id ORDER BY year, month) as prev_month_revenue
    FROM monthly_summary
    WHERE monthly_revenue > 0
)
SELECT 
    location_id,
    city,
    COUNT(*) as months_tracked,
    ROUND(AVG(monthly_revenue), 2) as avg_monthly_revenue,
    ROUND(AVG(CASE 
        WHEN prev_month_revenue > 0 THEN 
            ((monthly_revenue - prev_month_revenue) * 100.0 / prev_month_revenue)
        ELSE NULL 
    END), 2) as avg_growth_rate_percent,
    MIN(monthly_revenue) as lowest_month,
    MAX(monthly_revenue) as highest_month
FROM monthly_data
GROUP BY location_id, city
HAVING months_tracked >= 12
ORDER BY avg_growth_rate_percent DESC
LIMIT 15;
"""

growth_analysis = pd.read_sql_query(query1, conn)
print("Top 15 Locations by Growth Rate:")
print(growth_analysis[['location_id', 'city', 'avg_growth_rate_percent', 'avg_monthly_revenue']])

# ADVANCED QUERY 2: Cost Optimization Analysis
print("\nQUERY 2: Cost Optimization Opportunities")
print("-" * 40)

query2 = """
SELECT 
    l.location_id,
    l.city,
    l.location_type,
    l.monthly_rent,
    SUM(ms.monthly_revenue) as total_revenue,
    SUM(ms.monthly_profit) as total_gross_profit,
    SUM(ms.net_profit) as total_net_profit,
    ROUND((l.monthly_rent * 24) * 100.0 / SUM(ms.monthly_revenue), 2) as rent_percentage_of_revenue,
    CASE 
        WHEN (l.monthly_rent * 24) * 100.0 / SUM(ms.monthly_revenue) > 15 THEN 'High Rent Burden'
        WHEN SUM(ms.net_profit) < 0 THEN 'Unprofitable'
        WHEN SUM(ms.monthly_revenue) < l.monthly_rent * 3 THEN 'Low Revenue'
        ELSE 'Healthy'
    END as cost_status,
    ROUND(SUM(ms.monthly_revenue) / 24.0, 2) as avg_monthly_revenue,
    ROUND(l.monthly_rent * 100.0 / (SUM(ms.monthly_revenue) / 24.0), 2) as rent_to_revenue_ratio
FROM locations l
JOIN monthly_summary ms ON l.location_id = ms.location_id
GROUP BY l.location_id, l.city, l.location_type, l.monthly_rent
ORDER BY rent_percentage_of_revenue DESC;
"""

cost_analysis = pd.read_sql_query(query2, conn)
print("Cost Analysis - Locations needing attention:")
problem_locations = cost_analysis[cost_analysis['cost_status'] != 'Healthy']
print(problem_locations[['location_id', 'city', 'cost_status', 'rent_percentage_of_revenue']].head(10))

# ADVANCED QUERY 3: Menu Mix Analysis
print("\nQUERY 3: Menu Mix Optimization")
print("-" * 40)

query3 = """
WITH category_performance AS (
    SELECT 
        category,
        COUNT(*) as total_orders,
        SUM(revenue) as total_revenue,
        SUM(profit) as total_profit,
        ROUND(AVG(profit), 2) as avg_profit_per_item,
        ROUND(SUM(profit) * 100.0 / SUM(revenue), 2) as profit_margin_percent
    FROM sales
    GROUP BY category
),
total_business AS (
    SELECT SUM(total_revenue) as overall_revenue FROM category_performance
)
SELECT 
    cp.category,
    cp.total_orders,
    cp.total_revenue,
    cp.total_profit,
    cp.avg_profit_per_item,
    cp.profit_margin_percent,
    ROUND(cp.total_revenue * 100.0 / tb.overall_revenue, 2) as revenue_contribution_percent,
    CASE 
        WHEN cp.profit_margin_percent > 30 AND cp.total_revenue * 100.0 / tb.overall_revenue > 15 THEN 'Star'
        WHEN cp.profit_margin_percent > 30 THEN 'High Margin'
        WHEN cp.total_revenue * 100.0 / tb.overall_revenue > 20 THEN 'High Volume'
        ELSE 'Review Needed'
    END as category_status
FROM category_performance cp
CROSS JOIN total_business tb
ORDER BY cp.total_profit DESC;
"""

menu_mix = pd.read_sql_query(query3, conn)
print("Menu Category Performance:")
print(menu_mix)

# ADVANCED QUERY 4: Weather Impact Analysis
print("\nQUERY 4: Weather Impact on Sales")
print("-" * 40)

query4 = """
SELECT 
    w.temp_category,
    w.is_rainy,
    CASE WHEN w.is_rainy = 1 THEN 'Rainy' ELSE 'Clear' END as weather_condition,
    COUNT(DISTINCT ds.date) as number_of_days,
    ROUND(AVG(ds.daily_revenue), 2) as avg_daily_revenue,
    ROUND(AVG(ds.daily_profit), 2) as avg_daily_profit,
    ROUND(AVG(ds.daily_items_sold), 2) as avg_items_sold,
    ROUND(AVG(w.weather_impact_score), 3) as avg_weather_score
FROM daily_summary ds
JOIN weather w ON ds.date = w.date AND ds.city = w.city
GROUP BY w.temp_category, w.is_rainy
ORDER BY avg_daily_revenue DESC;
"""

weather_impact = pd.read_sql_query(query4, conn)
print("Weather Impact on Sales:")
print(weather_impact[['temp_category', 'weather_condition', 'avg_daily_revenue', 'avg_weather_score']])

# ADVANCED QUERY 5: Location Benchmarking
print("\nQUERY 5: Location Performance Benchmarking")
print("-" * 40)

query5 = """
WITH location_metrics AS (
    SELECT 
        ds.location_id,
        ds.city,
        ds.location_type,
        AVG(ds.daily_revenue) as avg_daily_revenue,
        AVG(ds.daily_profit) as avg_daily_profit,
        AVG(ds.daily_items_sold) as avg_daily_items,
        COUNT(*) as days_operating
    FROM daily_summary ds
    GROUP BY ds.location_id, ds.city, ds.location_type
),
benchmarks AS (
    SELECT 
        location_type,
        AVG(avg_daily_revenue) as benchmark_revenue,
        AVG(avg_daily_profit) as benchmark_profit,
        AVG(avg_daily_items) as benchmark_items
    FROM location_metrics
    GROUP BY location_type
)
SELECT 
    lm.location_id,
    lm.city,
    lm.location_type,
    lm.avg_daily_revenue,
    b.benchmark_revenue,
    ROUND((lm.avg_daily_revenue - b.benchmark_revenue) * 100.0 / b.benchmark_revenue, 2) as revenue_vs_benchmark_percent,
    lm.avg_daily_profit,
    b.benchmark_profit,
    ROUND((lm.avg_daily_profit - b.benchmark_profit) * 100.0 / b.benchmark_profit, 2) as profit_vs_benchmark_percent,
    CASE 
        WHEN lm.avg_daily_revenue > b.benchmark_revenue * 1.15 THEN 'Above Benchmark'
        WHEN lm.avg_daily_revenue < b.benchmark_revenue * 0.85 THEN 'Below Benchmark' 
        ELSE 'At Benchmark'
    END as performance_category
FROM location_metrics lm
JOIN benchmarks b ON lm.location_type = b.location_type
ORDER BY revenue_vs_benchmark_percent DESC
LIMIT 20;
"""

benchmarking = pd.read_sql_query(query5, conn)
print("Top/Bottom Performers vs Benchmark:")
print(benchmarking[['location_id', 'city', 'location_type', 'revenue_vs_benchmark_percent', 'performance_category']].head(10))

print("\nWorst Performers:")
print(benchmarking[['location_id', 'city', 'location_type', 'revenue_vs_benchmark_percent', 'performance_category']].tail(5))

print("\nSaving Advanced Analysis Results...")

# Save results
growth_analysis.to_csv('growth_analysis.csv', index=False)
cost_analysis.to_csv('cost_optimization.csv', index=False)
menu_mix.to_csv('menu_mix_analysis.csv', index=False)
weather_impact.to_csv('weather_impact.csv', index=False)
benchmarking.to_csv('performance_benchmarking.csv', index=False)

print("All advanced analysis results saved!")


ChainEats Advanced Analytics - Day 4

QUERY 1: Monthly Growth Analysis
----------------------------------------
Top 15 Locations by Growth Rate:
   location_id         city  avg_growth_rate_percent  avg_monthly_revenue
0      LOC_014      Chicago                     2.49            150268.36
1      LOC_041        Miami                     2.38             48825.96
2      LOC_043  Los Angeles                     2.38            194388.64
3      LOC_049      Houston                     2.36             84647.50
4      LOC_009        Miami                     2.29             61426.54
5      LOC_013      Houston                     2.24             55476.70
6      LOC_012      Houston                     2.23            107767.62
7      LOC_036      Chicago                     2.20            231579.11
8      LOC_037        Miami                     2.15             69380.12
9      LOC_023      Houston                     2.14             44900.36
10     LOC_035      Chicago              

In [3]:
# ChainEats Analytics - Day 4: Business Insights & Action Plan
import pandas as pd
import sqlite3

print("ChainEats Business Insights & Action Plan")
print("=" * 50)

# Load analysis results
growth_analysis = pd.read_csv('growth_analysis.csv')
cost_analysis = pd.read_csv('cost_optimization.csv')
menu_mix = pd.read_csv('menu_mix_analysis.csv')
weather_impact = pd.read_csv('weather_impact.csv')
benchmarking = pd.read_csv('performance_benchmarking.csv')

# INSIGHT 1: Growth Opportunities
print("INSIGHT 1: Growth Opportunities")
print("-" * 40)

top_growers = growth_analysis.head(5)
negative_growth = growth_analysis[growth_analysis['avg_growth_rate_percent'] < 0]

print("Fastest Growing Locations:")
for _, loc in top_growers.iterrows():
    print(f"  {loc['location_id']} ({loc['city']}): {loc['avg_growth_rate_percent']:.1f}% monthly growth")

print(f"\nDeclining Locations: {len(negative_growth)} locations with negative growth")
if len(negative_growth) > 0:
    worst_decline = negative_growth.tail(1).iloc[0]
    print(f"  Worst: {worst_decline['location_id']} ({worst_decline['city']}): {worst_decline['avg_growth_rate_percent']:.1f}%")

# INSIGHT 2: Cost Optimization Priorities
print("\nINSIGHT 2: Cost Optimization Priorities")
print("-" * 40)

high_rent_burden = cost_analysis[cost_analysis['cost_status'] == 'High Rent Burden']
unprofitable = cost_analysis[cost_analysis['cost_status'] == 'Unprofitable']
healthy_locations = cost_analysis[cost_analysis['cost_status'] == 'Healthy']

print(f"High Rent Burden: {len(high_rent_burden)} locations (>15% of revenue on rent)")
print(f"Unprofitable: {len(unprofitable)} locations")
print(f"Healthy Operations: {len(healthy_locations)} locations")

if len(high_rent_burden) > 0:
    worst_rent = high_rent_burden.iloc[0]
    print(f"\n  Worst rent burden: {worst_rent['location_id']} ({worst_rent['city']})")
    print(f"    Rent: ${worst_rent['monthly_rent']:,.0f}/month ({worst_rent['rent_percentage_of_revenue']:.1f}% of revenue)")

# INSIGHT 3: Menu Strategy
print("\nINSIGHT 3: Menu Strategy Recommendations")
print("-" * 40)

star_categories = menu_mix[menu_mix['category_status'] == 'Star']
review_needed = menu_mix[menu_mix['category_status'] == 'Review Needed']

print("Star Categories (High Margin + High Volume):")
for _, cat in star_categories.iterrows():
    print(f"  {cat['category']}: {cat['profit_margin_percent']:.1f}% margin, {cat['revenue_contribution_percent']:.1f}% of revenue")

print("\nCategories Needing Review:")
for _, cat in review_needed.iterrows():
    print(f"  {cat['category']}: {cat['profit_margin_percent']:.1f}% margin, {cat['revenue_contribution_percent']:.1f}% of revenue")

# INSIGHT 4: Weather Strategy
print("\nINSIGHT 4: Weather-Based Operations")
print("-" * 40)

best_weather = weather_impact.loc[weather_impact['avg_daily_revenue'].idxmax()]
worst_weather = weather_impact.loc[weather_impact['avg_daily_revenue'].idxmin()]

print(f"Best weather for sales: {best_weather['temp_category']} + {best_weather['weather_condition']}")
print(f"Average daily revenue: ${best_weather['avg_daily_revenue']:,.0f}")

print(f"Worst weather for sales: {worst_weather['temp_category']} + {worst_weather['weather_condition']}")
print(f"Average daily revenue: ${worst_weather['avg_daily_revenue']:,.0f}")

revenue_drop = ((best_weather['avg_daily_revenue'] - worst_weather['avg_daily_revenue']) / best_weather['avg_daily_revenue']) * 100
print(f"Weather impact: Up to {revenue_drop:.1f}% revenue difference")

# INSIGHT 5: Performance Benchmarking
print("\nINSIGHT 5: Performance vs Benchmarks")
print("-" * 40)

above_benchmark = benchmarking[benchmarking['performance_category'] == 'Above Benchmark']
below_benchmark = benchmarking[benchmarking['performance_category'] == 'Below Benchmark']

print(f"Above Benchmark: {len(above_benchmark)} locations")
print(f"Below Benchmark: {len(below_benchmark)} locations")

if len(above_benchmark) > 0:
    top_performer = above_benchmark.iloc[0]
    print(f"\n  Top Performer: {top_performer['location_id']} ({top_performer['city']})")
    print(f"    {top_performer['revenue_vs_benchmark_percent']:+.1f}% vs benchmark")

if len(below_benchmark) > 0:
    worst_performer = below_benchmark.tail(1).iloc[0]
    print(f"\n  Needs Attention: {worst_performer['location_id']} ({worst_performer['city']})")
    print(f"    {worst_performer['revenue_vs_benchmark_percent']:+.1f}% vs benchmark")

# COMPREHENSIVE ACTION PLAN
print("\n\nCOMPREHENSIVE ACTION PLAN")
print("=" * 50)

print("IMMEDIATE ACTIONS (Next 30 days):")
print("1. Review locations with high rent burden (>15% of revenue)")
print("2. Investigate declining locations for operational issues")
print("3. Promote star menu categories in marketing")

print("\nSHORT-TERM OPTIMIZATIONS (Next 90 days):")
print("4. Implement weather-based staffing adjustments")
print("5. Menu optimization - reduce low-margin items")
print("6. Benchmark training for underperforming locations")

print("\nSTRATEGIC DECISIONS (Next 6 months):")
print("7. Consider closing unprofitable locations")
print("8. Expand successful location types in high-growth cities")
print("9. Renegotiate rent for high-burden locations")

# Create summary metrics for executive report
summary_metrics = {
    'total_locations': len(cost_analysis),
    'profitable_locations': len(cost_analysis[cost_analysis['total_net_profit'] > 0]),
    'locations_above_benchmark': len(above_benchmark),
    'high_growth_locations': len(growth_analysis[growth_analysis['avg_growth_rate_percent'] > 2]),
    'star_menu_categories': len(star_categories),
    'weather_revenue_impact_percent': round(revenue_drop, 1)
}

print("\nEXECUTIVE SUMMARY METRICS:")
profitability_rate = (summary_metrics['profitable_locations'] / summary_metrics['total_locations']) * 100
print(f"  Profitability Rate: {profitability_rate:.1f}% ({summary_metrics['profitable_locations']}/{summary_metrics['total_locations']})")

benchmark_rate = (summary_metrics['locations_above_benchmark'] / summary_metrics['total_locations']) * 100  
print(f"  Above Benchmark: {benchmark_rate:.1f}% of locations")

print(f"  High Growth Locations: {summary_metrics['high_growth_locations']}")
print(f"  Star Menu Categories: {summary_metrics['star_menu_categories']}")
print(f"  Weather Impact: Up to {summary_metrics['weather_revenue_impact_percent']}% revenue variation")

# Save executive summary
pd.DataFrame([summary_metrics]).to_csv('executive_summary_metrics.csv', index=False)

print("\nAdvanced analytics done, ready for visualizations.")

ChainEats Business Insights & Action Plan
INSIGHT 1: Growth Opportunities
----------------------------------------
Fastest Growing Locations:
  LOC_014 (Chicago): 2.5% monthly growth
  LOC_041 (Miami): 2.4% monthly growth
  LOC_043 (Los Angeles): 2.4% monthly growth
  LOC_049 (Houston): 2.4% monthly growth
  LOC_009 (Miami): 2.3% monthly growth

Declining Locations: 0 locations with negative growth

INSIGHT 2: Cost Optimization Priorities
----------------------------------------
High Rent Burden: 1 locations (>15% of revenue on rent)
Unprofitable: 0 locations
Healthy Operations: 49 locations

  Worst rent burden: LOC_021 (Chicago)
    Rent: $5,981/month (16.2% of revenue)

INSIGHT 3: Menu Strategy Recommendations
----------------------------------------
Star Categories (High Margin + High Volume):
  Pizza: 63.5% margin, 31.9% of revenue
  Salads: 64.3% margin, 23.3% of revenue
  Burgers: 53.6% margin, 23.3% of revenue

Categories Needing Review:

INSIGHT 4: Weather-Based Operations
---